In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-11-03 14:52:54--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  67.2MB/s    in 0.4s    

2022-11-03 14:52:55 (67.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

**Explore the datasets**

In [4]:
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:
df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
df_ratings.isna().sum().sum()

0

In [7]:
df_books.isna().sum().sum()

1

In [8]:
df_books.dropna(inplace=True)

Aggregate the data

In [9]:
df_user_rating_count = (
    df_ratings.
    groupby('user')['rating']
    .count()
    .reset_index()
    .rename(columns = {'rating':'userTotalRatingCount'})
)

df_user_rating_count.sort_values(by="userTotalRatingCount", ascending=False).head()

,user,userTotalRatingCount
4213,11676,13602
74815,198711,7550
58113,153662,6109
37356,98391,5891
13576,35859,5850


In [10]:
df_book_rating_count = (
    df_ratings.
    groupby('isbn')['rating']
    .count()
    .reset_index()
    .rename(columns = {'rating':'bookTotalRatingCount'})
)

df_book_rating_count.sort_values(by="bookTotalRatingCount").tail()

,isbn,bookTotalRatingCount
41007,0312195516,723
9637,0060928336,732
83359,0385504209,883
47371,0316666343,1295
247408,0971880107,2502


Merge `df_book_rating_count`, `df_user_rating_count` and `df_ratings` into one dataset.

In [11]:
df_ratings = df_ratings.merge(df_book_rating_count, how='left', left_on='isbn', right_on="isbn")

df_ratings = df_ratings.merge(df_user_rating_count, how='left', left_on='user', right_on="user")

Drop statistically insignificant rows

In [12]:
mask_1 = df_ratings["bookTotalRatingCount"] > 100
mask_2 = df_ratings["userTotalRatingCount"] > 200

df_ratings = df_ratings[mask_1 & mask_2]

# Test

df_ratings.sort_values(by="userTotalRatingCount").head()

,user,isbn,rating,bookTotalRatingCount,userTotalRatingCount
1119534,268622,0553268880,0.0,265,201
125025,28634,0446525502,10.0,105,201
125024,28634,044651652X,10.0,368,201
125023,28634,0446364800,0.0,129,201
125021,28634,0440241537,10.0,241,201


Merge `df_ratings` with `df_books` into one dataset.

In [13]:
df_book_ratings =  df_ratings.merge(df_books, on="isbn")

df_book_ratings.head()

,user,isbn,rating,bookTotalRatingCount,userTotalRatingCount,title,author
0,277427,002542730X,10.0,171,497,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1,3363,002542730X,0.0,171,901,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
2,11676,002542730X,6.0,171,13602,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
3,12538,002542730X,10.0,171,1351,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
4,13552,002542730X,0.0,171,709,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner


In [14]:
df_book_ratings.duplicated().sum()

0

There are no duplicates

In [15]:
## Create a Pivot table

book_ratings_pivot = df_book_ratings.pivot_table(index='title',columns='user',values='rating').fillna(0)
book_ratings_pivot.head()

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Convert to 2D matrıx
book_ratings_matrix = csr_matrix(book_ratings_pivot.values)
book_ratings_matrix

<664x882 sparse matrix of type '<class 'numpy.float32'>'
	with 12327 stored elements in Compressed Sparse Row format>

**Build Model**

In [17]:
model_knn = NearestNeighbors(metric='cosine', algorithm="auto")
model_knn.fit(book_ratings_matrix)

NearestNeighbors(metric='cosine')

In [18]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):

  X = book_ratings_pivot[book_ratings_pivot.index == book]
  X = X.to_numpy().reshape(1,-1)
  distances, indices = model_knn.kneighbors(X,n_neighbors=8)
  recommended_books = []
  for x in reversed(range(1,6)):
      bookrecommended = [book_ratings_pivot.index[indices.flatten()[x]], distances.flatten()[x]]
      recommended_books.append(bookrecommended)
  recommended_books = [book, recommended_books]

  return recommended_books

In [19]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7230184]]]
You passed the challenge! 🎉🎉🎉🎉🎉
